## Authors: Oguzhan Ugur, Shadman Ahmed
## Mining datastream- "TRIEST: Counting Local and Global Triangles in Fully-Dynamic Streams with Fixed Memory Size"


### Problem Formulation
Webscaled networks are highly dynamic as new vertices can arrive and leave at any time of the graphical model. That is why we need an efficient algorithm to keep track of the properties of the evolving network. Through edge insertions. 

TRIEST provides an unbiased estimator for the total number of global and local triangles over an evolving streaming graph. And maintains a small sample subgraph of a fixed size.

In this homework we were instructed to choose one of three papers and implement the algorithm introduced in that paper. We chose to implement the Triésta-base and Triésta algorithm of the paper _"TRIÉSTA: Counting Local and Global Triangles in Fully-Dynamic Streams with Fixed Memory Size"_. The problems we had to deal with was,
- First, Implement reservoir sampling in order to exploit the available memory. 
- Second, Implement the Triésta base algorithm, that counts the triangles.
- Third, Implement the Triesta improved algorithm, to get lower variance estimates.

### Dataset

This network contains of links between friends and family for each users of the website hamsterster.com. In total there are: 2,426 vertices (users), 16,631 edges (friendships) and 53,265 triangles.   

### TRIEST

The baseline algorithm of TRIEST maintains a collection S of M edges from the stream. Randomly samples (reservoir sample) an edge from S with and replace it with an edge from the stream. Then decrease or increase the number of triangle counts involved with deletion or insertion of the new edge into S. The improved TRIEST algorithm is different from the basic one in terms of updating the count, where it weighs the counting and some other minor modifications resulting in lower variance estimations of the triangle count.


In [ ]:
#import libraries
import random 
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt

In [ ]:
def extract_from_file(file,line_skips):
    with open(file, "r") as f:
        for _ in range(line_skips): #skips the # first lines in file
            next(f)
        for edge in f:
            edge = edge.split()
            edge_tuple = tuple(map(int, edge))
                             
            yield edge_tuple #('+', edge_tuple)

# Triést basic algorithm
#### code 

The TRIEST_BASE class consist of 6 functions, 

- First, The _FlipBiasedCoin_ function that generates a random number and returns true if a given number Hprob is greater than the generated value. 
- Second, _SampleEdge_ function that looks into the memory M and the time step t. This function returns true if M is greater or equal to t, else it uses the _FlipBiasedCoin_ function that returns true or false based on the input given to it. If it returns true then a random edge from the edge sample,"Sample\_S", is deleted and the counters decremented. If the _FlipBiasedCoin_ returns False then this function will return False as well. 
- Third, _find\_neighb_ function finds the neighbours for each node and computes the intersection of the neighbours for two nodes that has an edge, in order to find common neighbours. 
- Fourth, _UpdateCounters_ is the function responsible of either decrementing or adding the counters. This function basically decrements the counters if a given sign is '-' and increases the counters if it is '+'.
- Fifth, _expected\_value_ computes the estimated number of triangles by using a theorem introduced in the paper for the Triést algorithm.
- Sixth, _run_ function that creates the algorithm by looking into the _SampleEdge_ function and if that function returns true then it will add the current edge to the edge sample and increase the counter. This function returns the local counter, global counter and the estimated value using the _expected\_value_ function. 

In [ ]:
class TRIEST_BASE:
    
    def __init__(self, M, file, skips):
        self.M = M
        self.stream = extract_from_file(file,skips)
        self.global_counter = 0
        self.local_counter = defaultdict(int)
        self.sample_S = set()
        self.t = 0
        
        
    def FlipBiasedCoin(self,Hprob):
        random.seed(100)
        coin = random.random()
        if Hprob > coin: #prob of getting head
            return (True)
    
    
    def SampleEdge(self,edge,t):
        if self.M >= self.t:
            return (True)
        elif self.FlipBiasedCoin(self.M/self.t):

            randSample = random.sample(self.sample_S,1)[0]
            self.sample_S.remove(randSample)
            self.UpdateCounters('-',randSample)
            
            return(True)
        
        return(False)
    
    def find_neighb(self,u,v):
        
        N_u =set()  ; N_v =set()
    
        for vertices in self.sample_S:
            if u == vertices[1]:
                N_u.add(vertices[0])
            elif u == vertices[0]:
                N_u.add(vertices[1])
            
            if v == vertices[1]:
                N_v.add(vertices[0])
            elif v == vertices[0]:
                N_v.add(vertices[1])
        N_uv = N_u & N_v
        
        return N_uv
    
    def delete_empty_count(self,counter,vertices):
        if(counter[vertices] == 0):
            del counter[vertices]
 

    def UpdateCounters(self,op, edge):
    
        u = edge[0] ; v = edge[1]
        N_uv = self.find_neighb(u,v)
    
        num_of_triangles = len(N_uv)


        if op == '+':
            self.global_counter = self.global_counter + num_of_triangles
            self.local_counter[u] = self.local_counter[u] + num_of_triangles
            self.local_counter[v] = self.local_counter[v] + num_of_triangles
            
            self.delete_empty_count(self.local_counter,u)
            self.delete_empty_count(self.local_counter,v)
            
            for c in N_uv:
                self.local_counter[c] = self.local_counter[c] + 1
                self.delete_empty_count(self.local_counter,c)
        
        elif op == '-':
            self.global_counter = self.global_counter - num_of_triangles
            self.local_counter[u] = self.local_counter[u] - num_of_triangles
            self.local_counter[v] = self.local_counter[v] - num_of_triangles
            
            self.delete_empty_count(self.local_counter,u)
            self.delete_empty_count(self.local_counter,v)
            
            for c in N_uv:
                self.local_counter[c] = self.local_counter[c] - 1
                self.delete_empty_count(self.local_counter,c)
                
    def expected_value(self,t,M,tau):
        if(M >= t):
            return tau
        else:
            epsilon = max(1,(self.t*(self.t-1)*(self.t-2))/(M*(M-1)*(M-2)))
            estimated_value = epsilon*tau
            return estimated_value
             
    def run(self):
        for edge in self.stream: #+ edge
            print("--edge--")
            print(edge)
            self.t = self.t + 1
            if self.SampleEdge(edge,self.t):
                self.sample_S.add(edge)
                self.UpdateCounters('+',edge)
                
        estimated = self.expected_value(self.t,self.M,self.global_counter)
                
        return(self.local_counter,self.global_counter,estimated)

In [ ]:
skip_lines = 16600
file_path = "out.petster-hamster"
memory = 20
Base_obj = TRIEST_BASE(memory , file_path, skip_lines)
local_count,glob_count,estimated = Base_obj.run()
print(estimated)

In [ ]:
M = [100, 1000, 1500, 2000, 2500, 3000, 3500, 6000, 9000, 13000]
count = []
for i in M:
    Base_obj = TRIEST_BASE(i, file_path, skip_lines)
    local_count,fin_obj,estimated = Base_obj.run()
    count.append(estimated)

# Triést improved algorithm

The Triést improved algorithm is a modified version of the Triést Base algorithm. The improvement of this algorithm results in lower variance estimations. This algorithm was modified by 
- First, changing the way the global counter was updated. The global counter is updated unconditionally for each element on the 'stream' before the algorithm decides whether or not to insert the edge in the edge sample, "sample_S".
- Second, not decrementing the counters when an edge is removed from the edge sample. 
- Third, using a weighted increase of the counters.

This algorithm basically works similar to the base algorithm but it doesn't decrement the counters and it uses weighted counters that takes the time step and memory into account. The _SampleEdge_ function is similar to the function in the base algorithm but the difference is that it never updates the counters in this function as it did before. 


In [ ]:
class TRIEST_IMPR:
    
    def __init__(self, M, file, skips):
        self.M = M
        self.stream = extract_from_file(file,skips)
        self.global_counter = 0
        self.local_counter = defaultdict(int)
        self.sample_S = set()
        self.t = 0
        
        
    def FlipBiasedCoin(self,Hprob):
        random.seed(100)
        coin = random.random()
        if Hprob > coin: #prob of getting head
            return (True)
    
    
    def SampleEdge(self,edge,t):
        if self.M >= self.t:
            return (True)
        elif self.FlipBiasedCoin(self.M/self.t):
            randSample = random.sample(self.sample_S,1)[0]
            self.sample_S.remove(randSample)
            
            return(True)
        
        return(False)
    
    def find_neighb(self,u,v):
        
        N_u =set()  ; N_v =set()
    
        for vertices in self.sample_S:
            if u == vertices[1]:
                N_u.add(vertices[0])
            elif u == vertices[0]:
                N_u.add(vertices[1])
            
            if v == vertices[1]:
                N_v.add(vertices[0])
            elif v == vertices[0]:
                N_v.add(vertices[1])

        N_uv = N_u & N_v  
        
        return N_uv
    
    def delete_empty_count(self,counter,vertices):
        if(counter[vertices] <= 0.0):
            del counter[vertices]
 

    def UpdateCounters(self, edge):
    
        u = edge[0] ; v = edge[1]
        N_uv = self.find_neighb(u,v)
    
        num_of_triangles = len(N_uv)
        
        eta = max(1, ((self.t-1)*(self.t))/(self.M*(self.M-1)))
            

        self.global_counter = self.global_counter + num_of_triangles*eta
        self.local_counter[u] = self.local_counter[u] + num_of_triangles*eta
        self.local_counter[v] = self.local_counter[v] + num_of_triangles*eta
            
        self.delete_empty_count(self.local_counter,u)
        self.delete_empty_count(self.local_counter,v)
            
        for c in N_uv:
            self.local_counter[c] = self.local_counter[c] + eta
            self.delete_empty_count(self.local_counter,c)
                
    def run(self):
        for edge in self.stream: #+ edge
            self.t += 1
            self.UpdateCounters(edge)
            if self.SampleEdge(edge,self.t):
                self.sample_S.add(edge)
                
                
        return(self.local_counter,self.global_counter)

In [ ]:
skip_lines = 1
file_path = "petster-hamster/out.petster-hamster"
memory = 17000

impr_obj = TRIEST_IMPR(memory, file_path, skip_lines)
local_count,glob_count= impr_obj.run()
print(glob_count)

In [ ]:
M = [100,1000, 1500, 2000, 2500, 3000, 3500, 6000, 9000, 13000]
count_impr = []
for i in M:
    Base_obj = TRIEST_IMPR(i, file_path, skip_lines)
    local_count,global_count = Base_obj.run()
    count_impr.append(global_count)

In [ ]:
plt.figure(figsize=(15,10))
plt.subplot(2,2,1)
plt.plot(M,count_impr)
plt.title('Triésta Improved')

plt.subplot(2,2,2)
plt.plot(M,count,'r')
plt.title('Triésta base')

### Result

As it is possible to see the improved version of the Triésta algorithm has lower variance since it varies between 225000 and 27000 while the base algorithm varies between 450000 and 50000. This is the result we wanted to see since the argument to improve the base algorithm was to reduce the variance for lower memories. 

# Optional task for extra bonus

- 1. The challenges we faced while implementing the algorithm were mostly to understand the theory. The implementing of the algorithm was not that hard. The part we got stuck on at the beggining were how to create the updatings of the counters. We didn't face any difficulties that was really hard to deal with. 
- 2. It is probably possible to parallelize the algorithm but that would lead to huge changes of the algorithm. The algorithm depends on a global counter that estimates the number of triangles. The global counter and the local counters are dependent of the information about the neighbourhood received from the nodes(vertices) of the edges. So, it is a process that goes hand in hand for the counters which would make it hard to parallelize. This means that workers has to communicate with each other in order to update and synchronize the counters for each time step. So, in other words it is difficult to parallelize this algorithm. 
- 3. The algorithm works with unbounded graph streams with help of the reservoir sampling method and maintaining a fixed memory space M. A unbounded stream can consequence in under-utilization of memory or memory shortage but in our case that is not possible since this algorithm does take the size/memory in to account.  
- 4. The TRIEST algorithms are for insertion-only streams. But the fully-dynamic algorithm handles streams of edges that are inserted or deleted in any arbitrary order with help of random pairing (RP), an extention of the reservoir sampling that can handel deletions of edges. The main idea is to compensate the future insertions of edges after a deletion of the edges has occured, by using a counter.